In [5]:
using Random
using DelimitedFiles
using SparseArrays
using LinearAlgebra

In [20]:
n = 24
deg_c = 4 # w_r. Every check has this many bits in it
deg_v = 3 # w_c. Every bit is in this many checks
num_checks = div(n*deg_v, deg_c)
k = n - num_checks

vs = vcat([[j for i in 1:deg_v] for j in 1:n]...)
cs = vcat([[j for i in 1:deg_c] for j in 1:num_checks]...)

H = zeros(Int8, num_checks, n)

while(length(vs) > 0 && length(cs) > 0)
    double_edge = true
    
    while(double_edge)
        shuffle!(vs)
        shuffle!(cs)

        if (H[cs[1], vs[1]] != 1)
            double_edge = false
            H[cs[1], vs[1]] = 1
            popfirst!(vs)
            popfirst!(cs)
        end
    end
end


In [21]:
hx1 = kron(H, Matrix{Int8}(I, size(H)[2], size(H)[2]));
hx2 = kron(Matrix{Int8}(I, size(H)[1], size(H)[1]), transpose(H))
Hx = sparse(hcat(hx1, hx2))

hz1 = kron(Matrix{Int8}(I, size(H)[2], size(H)[2]), H)
hz2 = kron(transpose(H), Matrix{Int8}(I, size(H)[1], size(H)[1]))
Hz = sparse(hcat(hz1, hz2))

432×900 SparseMatrixCSC{Int8, Int64} with 3024 stored entries:
⠛⣧⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⢤⠀⢀⢤⡻⣗⠀⠀⠀⠀⣙⠂⠀
⠀⠀⠙⠷⣦⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠌⠣⣄⠸⠇⠀⠀⠀⣄⠀⠐⠎⣇⠀
⠀⠀⠀⠀⠈⠙⢻⣦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⠒⢾⡄⠀⠑⠀⠀⣈⢧⢀⠀⣈⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠹⢷⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⢘⠆⢤⡀⢠⣤⡈⠙⠮⠁⠈⡳
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠻⢶⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⠇⢦⡈⠃⠀⢀⡀⠀⠀⠦⢀⡀⠀⠙
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠿⣤⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⡎⠻⠆⠀⠀⠩⣤⡀⢄⠀⠁⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠷⣦⡄⠀⠀⠀⠀⠀⠀⠲⠝⠀⠀⠀⣄⠀⠈⠃⠀⠑⢤⡲⣦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠛⣷⣄⡀⠀⠀⠀⠀⢦⠀⢘⠛⢊⡰⡄⠀⢰⣄⠀⠙⠈
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠿⣦⡄⠀⠀⠀⠀⠈⠁⣄⢩⡀⠳⣄⠙⠷⠄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠋⠀⠀⠀⠀⠙⠂⠀⠙⠛⠂⠀⠀⠀⠀

In [92]:
function powerset(x::Vector{T}) where T
    result = Vector{T}[[]]
    for elem in x, j in eachindex(result)
        push!(result, [result[j] ; elem])
    end
    return result
end

function syn_from_F(F, H)
    result = Set()
    for i in 1:size(Hx)[1]
        row = Set(H[i, :].nzind)
        if length(intersect(row, F)) > 0
            push!(result, i)
        end
    end
    return result
end

function syn_from_F2(F, H)
    z = zeros(Int8, size(H)[2])
    z[collect(F)] = ones(length(F))
    # e = findnz(sparse((H * z) % 2))
    # (H * z) % 2
    return Set()
end

Fx = vcat([powerset(findnz(sparse(Hx)[i, :])[1])[2:end] for i in 1:size(Hx)[1]]...);
Fx = map(x -> Set(x), Fx)
Fz = vcat([powerset(findnz(sparse(Hz)[i, :])[1])[2:end] for i in 1:size(Hz)[1]]...);
Fz = map(x -> Set(x), Fz)

# sigma_Fx = [syn_from_F(x, Hx) for x in Fx]  # set of indices where syndrome is 1
# sigma_Fz = [syn_from_F(g, Hz) for g in Fz];

In [136]:
intersect(Set(findnz(sparse(Hx)[1, :])[1]), Set(Fx[1]))

Set{Int64} with 1 element:
  17

In [93]:
@time syn_from_F2(Fx[1], Hx)

  0.042854 seconds (12.23 k allocations: 631.097 KiB, 99.88% compilation time)


Set{Any}()

In [86]:
@time syn_from_F(Fx[1], Hx)

  0.036559 seconds (20.83 k allocations: 1.315 MiB, 92.15% compilation time)


Set{Any} with 3 elements:
  121
  337
  1

In [74]:
Fx[1]

Set{Int64} with 1 element:
  73

In [68]:
z = zeros(Int8, size(Hx)[2])
z[collect(Fx[1])] = ones(length(Fx[1]))

1-element Vector{Float64}:
 1.0

In [71]:
findnz(sparse(z))

([73], Int8[1])

In [66]:
print((Hx * z) .% 2)

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]